<div dir="rtl" style="width:90%; margin:auto">
<p>
فشرده‌سازی داده‌ها فرآیندی است که در آن حجم داده‌ها کاهش می‌یابد تا فضای کمتری اشغال شود و انتقال آن‌ها سریع‌تر انجام شود. این فرآیند در بسیاری از حوزه‌ها و کاربردها اهمیت دارد:
</p>

<ul>
<li><strong>ذخیره‌سازی داده‌ها: </strong></li>
<li><strong>انتقال داده‌ها: </strong></li>
<li><strong>پایگاه‌های داده: </strong></li>
<li><strong>پایگاه‌های داده: </strong></li>
<li><strong>توزیع نرم‌افزار: </strong></li>
<li><strong>آرشیو کردن: </strong></li>
<li><strong>شتیبان‌گیری (Backup): </strong></li>
</ul>
<p>زمانی که با stream ها کار می کنیم، بحث فشرده سازی مطرح است. سی شارپ در فضای نام <code>System.IO.Compression</code> دو stream فشرده سازی عمومی دارد دارد</p>

<ol><li><p><strong>DeflateStream</strong>:</p><ul><li>این استریم از الگوریتم فشرده‌سازی پرکاربردی استفاده می‌کند که مشابه فرمت ZIP است.</li><li>این استریم به سادگی داده‌ها را فشرده و یا از حالت فشرده خارج می‌کند، بدون اضافه کردن اطلاعات اضافی.</li></ul></li><li><p><strong>GZipStream</strong>:</p><ul><li>این استریم نیز از همان الگوریتم فشرده‌سازی DeflateStream استفاده می‌کند.</li><li>تفاوت اصلی GZipStream این است که در ابتدای داده‌های فشرده‌شده و در انتهای آن یک پروتکل اضافی اضافه می‌کند که شامل یک CRC (Cyclic Redundancy Check) برای تشخیص خطاها می‌باشد.</li><li>GZipStream به یک استاندارد شناخته‌شده توسط نرم‌افزارهای دیگر پایبند است، بنابراین داده‌های فشرده‌شده با این استریم می‌توانند توسط نرم‌افزارهای دیگر نیز به راحتی خوانده شوند.</li></ul></li></ol>

<h4>BrotliStream</h4>
<p>.NET همچنین شامل <strong>BrotliStream</strong> است که الگوریتم فشرده‌سازی Brotli را پیاده‌سازی می‌کند.</p>
<ul>
<li>
<strong>BrotliStream</strong>:
<ul>
<li>این استریم از الگوریتم فشرده‌سازی Brotli استفاده می‌کند که نسبت به DeflateStream و GZipStream فشرده‌سازی بهتری ارائه می‌دهد.</li>
<li>با این حال، فشرده‌سازی با BrotliStream بیش از ۱۰ برابر کندتر از DeflateStream و GZipStream است.</li>
<li>این کاهش سرعت فقط در فرآیند فشرده‌سازی اتفاق می‌افتد و در فرآیند خارج کردن از حالت فشرده عملکرد بسیار خوبی دارد.</li>
<li>
BrotliStream با ارائه نرخ فشرده‌سازی بهتر، انتخاب مناسبی برای زمانی است که حجم فایل نهایی اهمیت بیشتری نسبت به سرعت فشرده‌سازی دارد. 
</li>
</ul></li></ul>
</div>

In [1]:
using System.IO;
using System.IO.Compression;

using (FileStream fs = new FileStream("file.txt", FileMode.Create))
using (GZipStream gzip = new GZipStream(fs, CompressionMode.Compress))
using (StreamWriter writer = new StreamWriter(gzip))
{
    writer.Write("This is a test.");
}

<div dir="rtl" style="width:90%; margin:auto">
<p>
داده‌ها ابتدا توسط StreamWriter نوشته می‌شوند، سپس توسط GZipStream فشرده می‌شوند و در نهایت در FileStream (یعنی فایل file.txt) ذخیره می‌شوند.
</p>
</div>

In [ ]:
// read from ziped File

using System.IO;
using System.IO.Compression;

using (FileStream fs = new FileStream("file.txt", FileMode.Open))
using (GZipStream gzip = new GZipStream(fs, CompressionMode.Decompress))
using (StreamReader reader = new StreamReader(gzip))
{
    string result = reader.ReadToEnd();
    Console.WriteLine(result);
}


<div dir="rtl" style="width:90%; margin:auto">
<h4>نحوه عملکرد فشرده‌سازی</h4>
<ul><li><strong>کاهش افزونگی</strong>: فشرده‌سازی با حذف یا کاهش داده‌های تکراری و الگوهای مشابه در داده‌ها، حجم آن‌ها را کاهش می‌دهد.</li><li><strong>کدگذاری بهینه</strong>: با استفاده از کدهای کوتاه‌تر برای داده‌های پرکاربردتر، حجم کل داده‌ها کاهش می‌یابد.</li><li><strong>بهره‌گیری از الگوها</strong>: فشرده‌سازی الگوهای تکراری و ساختارهای مشابه در داده‌ها را شناسایی کرده و به صورت بهینه ذخیره می‌کند.</li></ul>


<p><strong>کدگذاری بر اساس طول اجرا (Run-Length Encoding - RLE)</strong>:</p>
<ul><li>این تکنیک به شناسایی و فشرده‌سازی رشته‌های طولانی از داده‌های تکراری می‌پردازد.</li><li>به جای ذخیره کردن هر کاراکتر تکراری به صورت جداگانه، تعداد تکرار و کاراکتر تکراری را ذخیره می‌کند.</li><li>مثال: رشته <code>AAAABBBCCDAA</code> به <code>4A3B2C1D2A</code> فشرده می‌شود.</li></ul>

<p><strong>فشرده‌سازی مبتنی بر دیکشنری</strong>:</p>
<ul><li>این روش به جای ذخیره‌سازی مستقیم داده‌ها، از دیکشنری‌هایی استفاده می‌کند که رشته‌های داده‌های تکراری را به مقادیر کوتاه‌تر ترجمه می‌کنند.</li><li>الگوریتم‌های معروفی مانند <strong>Lempel-Ziv-Welch (LZW)</strong> و <strong>Lempel-Ziv (LZ77)</strong> از این تکنیک استفاده می‌کنند.</li><li>مثال: <code>ABABABAB</code> به <code>AB(AB)*3</code> فشرده می‌شود.</li></ul>

<p><strong>کدگذاری هافمن (Huffman Encoding)</strong>:</p>
<ul><li>این تکنیک از یک درخت دودویی برای کدگذاری داده‌ها استفاده می‌کند که در آن کاراکترهای پرکاربردتر کدهای کوتاه‌تری دارند.</li><li>به این ترتیب، مجموع حجم داده‌های فشرده‌شده کاهش می‌یابد.</li><li>مثال: در یک متن انگلیسی، فاصله (space) که بیشتر تکرار می‌شود، کد کوتاه‌تری دارد.</li></ul>

<h4>الگوریتم‌های معروف فشرده‌سازی</h4>
<ol><li><p><strong>Deflate</strong>:</p><ul><li>ترکیبی از الگوریتم‌های LZ77 و هافمن است.</li><li>در فشرده‌سازی‌های ZIP و GZip استفاده می‌شود.</li><li>کارایی خوبی در فشرده‌سازی و استخراج دارد.</li></ul></li><li><p><strong>GZip</strong>:</p><ul><li>از الگوریتم Deflate استفاده می‌کند اما با اضافه کردن هدر و CRC برای تشخیص خطا.</li><li>یک استاندارد شناخته‌شده برای فشرده‌سازی فایل‌ها است.</li></ul></li><li><p><strong>Brotli</strong>:</p><ul><li>الگوریتمی که توسط Google توسعه یافته و فشرده‌سازی بالاتری نسبت به Deflate دارد.</li><li>به خصوص برای فشرده‌سازی محتوای وب بسیار موثر است.</li></ul></li><li><p><strong>Bzip2</strong>:</p><ul><li>از فشرده‌سازی مبتنی بر Burrows-Wheeler و کدگذاری هافمن استفاده می‌کند.</li><li>نسبت به Deflate فشرده‌سازی بهتری ارائه می‌دهد اما کندتر است.</li></ul></li></ol>
</div>

`DeflateStream`, `GZipStream`, and `BrotliStream` are ***decorators***; they **compress** or **decompress** data from another stream that you supply in construction.

In [3]:
using System.IO;
using System.IO.Compression;

string[] words = "The quick brown fox jumps over the lazy dog".Split();
Random rand = new Random (0); // Give it a seed for consistency

using (Stream s = File.Create ("compressed.bin"))
using (Stream ds = new BrotliStream (s, CompressionMode.Compress))
using (TextWriter w = new StreamWriter (ds))
    for (int i = 0; i < 1000; i++)
        await w.WriteAsync (words [rand.Next (words.Length)] + " ");

Console.WriteLine (new FileInfo ("compressed.bin").Length); 


using (Stream s = File.OpenRead ("compressed.bin"))
using (Stream ds = new BrotliStream (s, CompressionMode.Decompress))
using (TextReader r = new StreamReader (ds))
    Console.Write ((await r.ReadToEndAsync()).Length); 

1062
4936

### Compressing in Memory

In [4]:
using System.IO;
using System.IO.Compression;

byte[] data = new byte[1000]; // We can expect a good compression
                                // ratio from an empty array!

var ms = new MemoryStream();
using (Stream ds = new DeflateStream (ms, CompressionMode.Compress))
    ds.Write (data, 0, data.Length);

byte[] compressed = ms.ToArray();
Console.WriteLine (compressed.Length); // 11

// Decompress back to the data array:
ms = new MemoryStream (compressed);
using (Stream ds = new DeflateStream (ms, CompressionMode.Decompress))
    for (int i = 0; i < 1000; i += ds.Read (data, i, 1000 - i));

11
